In [ ]:
import pandas as pd
import json

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

with open('en_data2.json') as json_file:
    data = json.load(json_file)


# Get HS
bst = pd.read_excel('BIEU-THUE-XNK-2021.06.22.xlsx', header=4, converters={'B':str, 'C':str, 'D': str})
bst = bst[['B', 'C', 'D']]

bst.drop(bst[pd.isna(bst['B']) & pd.isna(bst['C'])].index, inplace = True)


def remove_useless_end(row):
    if str(row['B']).endswith('00'):
        return str(row['B'])[:-2]
    else:
        return row['B']

bst['B'] = bst.apply(lambda row: remove_useless_end(row), axis=1)

bst['temp'] = bst['B'].shift(-1)
bst['temp'] = bst['temp'].astype(str)


def remove_useless_temp(row):
    if str(row['temp']).endswith('0'):
        return str(row['temp'])[:-1]
    else:
        return row['temp']

bst['temp'] = bst.apply(lambda row: remove_useless_temp(row), axis=1)


def get_hs5(row):
    result = row['B']
    # if re.match('- ([a-zA-Z0-9])\w+', row['D']) and pd.isna(row['B']):
    #     result = row['temp'][0:5]
    if str(row['D']).startswith('- ') and pd.isna(row['B']):
        result = row['temp'][0:-1]
    else:
        result = row['B']
    return result

bst['B'] = bst.apply(lambda row: get_hs5(row), axis=1)


In [ ]:
headings_lst = [heading for heading in bst['B'].unique() if len(str(heading)) == 4]

In [ ]:
for section in data:
    for chapter in section['chapters']:
        for heading in chapter['headings']:
            for hs6 in heading['hs6']:
                hs6['id'] = hs6['id'].split('-')[0]
                if hs6['id'][4] != '.':
                    hs6['id'] = hs6['id'][:4] + '.' + hs6['id'][4:]
                try:
                    hs_to_lookup = ''.join(hs6['id'].split('.'))
                    hs_to_lookup = ''.join(hs_to_lookup.split(' '))
                    hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup, ['C']].iloc[0]['C']
                except IndexError:
                    try:
                        hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup + '0', ['C']].iloc[0]['C']
                    except IndexError:
                        if hs_to_lookup.endswith('-') or hs_to_lookup.endswith('0'):
                            try:
                                hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup[:-1], ['C']].iloc[0]['C']
                            except IndexError:
                                print(hs_to_lookup)
                        else:
                            try:
                                hs6['title'] = bst.loc[(bst['B']) == hs_to_lookup.split('-')[0], ['C']].iloc[0]['C']
                            except IndexError:
                                print(hs_to_lookup)
                